In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
df_supply = pd.read_csv('out_input_MS-m1-supply.csv')
df_supply.head()

,date,apple,pen
0,2006-01-01,15057,810
1,2006-01-15,15001,752
2,2006-02-01,15144,783
3,2006-02-15,14871,802
4,2006-03-01,14979,804


In [3]:
df_inventory = pd.read_csv('out_input_MS-m1-inventory.csv')
df_inventory.head()

,date,apple,pen
0,2006-01-31,3678,122
1,2006-02-28,9921,491
2,2006-03-31,13607,766
3,2006-04-30,17930,824
4,2006-05-31,21458,993


In [4]:
df_sell = pd.read_csv('out_input_MS-m1-sell.csv')
df_sell.head()

,date,sku_num
0,2006-01-01,MS-m1-ap-9974cfbb-246e-4d87-a26b-62e425082d99
1,2006-01-01,MS-m1-ap-ae8d6d6e-8018-486d-ab6c-f01dda14fd8e
2,2006-01-01,MS-m1-ap-5b9ab304-2819-46d4-a761-fb4e0632c3ae
3,2006-01-01,MS-m1-ap-4018b624-b1c7-48ad-b7a2-d0e759ee4e12
4,2006-01-01,MS-m1-ap-e6dcf9fd-ab3e-4779-8039-dbe04db81df0


In [5]:
state = df_sell.sku_num[0][:2]

In [6]:
total_apples = 0
total_pens = 0
res_df = pd.DataFrame(columns=['date', 'apple', 'pen'])

from_ind = 0
to_ind = 0
prev_date = '2006-01-01'

for cur_sell in df_sell.values:
    str_d = cur_sell[0]
    code = cur_sell[1][6:8]
    
    if prev_date != str_d:
        sup = df_supply[df_supply.date==prev_date]
        
        if sup.shape[0] == 1:
            total_apples += int(sup.apple)
            total_pens += int(sup.pen)

        cur_df = pd.DataFrame(columns=['date', 'apple', 'pen'], data=[[prev_date, total_apples, total_pens]])
        res_df = res_df.append(cur_df)
    
    prev_date = str_d

    if code == 'ap':
        total_apples -= 1
    else:
        total_pens -= 1


sup = df_supply[df_supply.date==str_d]
if sup.shape[0] == 1:
    total_apples += int(sup.apple)
    total_pens += int(sup.pen)
cur_df = pd.DataFrame(columns=['date', 'apple', 'pen'], data=[[str_d, total_apples, total_pens]])
res_df = res_df.append(cur_df)
res_df.head()

,date,apple,pen
0,2006-01-01,14244,767
0,2006-01-02,13395,722
0,2006-01-03,12475,689
0,2006-01-04,11601,634
0,2006-01-05,10775,586


In [7]:
stolen_df = pd.DataFrame(columns=['date', 'apple', 'pen'])

total_stolen_apples = 0
total_stolen_pens = 0

for x in df_inventory.values:
    cur_d = x[0]
    paper_apples = x[1]
    paper_pens = x[2]
    real_inv = res_df[res_df.date==x[0]]
    real_apples = int(real_inv.apple)
    real_pens = int(real_inv.pen)
    cur_apples = real_apples-paper_apples-total_stolen_apples
    cur_pens = real_pens-paper_pens-total_stolen_pens
    cur_df = pd.DataFrame(columns=['date', 'apple', 'pen'], data=[[cur_d, cur_apples, cur_pens]])
    total_stolen_apples += cur_apples
    total_stolen_pens += cur_pens
    stolen_df = stolen_df.append(cur_df)
stolen_df.head()

,date,apple,pen
0,2006-01-31,6,9
0,2006-02-28,2,5
0,2006-03-31,7,5
0,2006-04-30,9,5
0,2006-05-31,4,8


In [25]:
cols = ['year', 'state', 'apple_sold', 'apple_stolen', 'pen_sold', 'pen_stolen']
year_df = pd.DataFrame(columns=cols)

cur_year = 2006
supply_pens = 0
supply_apples = 0
ind = 0

for x in df_supply.values:
    cur_date = x[0]
    supply_apples += x[1]
    supply_pens += x[2]
    
    if ind % 24  == 23:
        cur_date = cur_date[:-2]+'31'
        apple_sold = supply_apples-int(res_df[res_df.date==cur_date].apple)
        pen_sold = supply_pens-int(res_df[res_df.date==cur_date].pen)
        pen_stolen = int(res_df[res_df.date==cur_date].pen)-int(df_inventory[df_inventory.date==cur_date].pen)
        apple_stolen = int(res_df[res_df.date==cur_date].apple)-int(df_inventory[df_inventory.date==cur_date].apple)
        cur_df = pd.DataFrame(columns=cols, data=[[cur_year, state, apple_sold, apple_stolen, pen_sold, pen_stolen]])
        year_df = year_df.append(cur_df)
        cur_year += 1
    ind += 1
year_df.head()

,year,state,apple_sold,apple_stolen,pen_sold,pen_stolen
0,2006,MS,309956,65,16178,79
0,2007,MS,619433,127,32367,131
0,2008,MS,930596,194,48508,189
0,2009,MS,1240175,258,64696,255
0,2010,MS,1550119,321,80632,313
